In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
logging.basicConfig()
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data
df = data('diamonds').reset_index().set_index('index')

import frankenfit as ff
from frankenfit import core as ffc
# reload(ffc)

logging.getLogger('frankenfit').setLevel(logging.INFO)

from typing import Optional, Any, TypeVar

INFO:__main__:Hello from notebook


In [2]:
i = ff.Identity[str]()
z = i.fit("x").apply("x")
y = i.apply("x")

In [2]:
@ff.params
class DeMean(ff.Transform[pd.DataFrame, pd.DataFrame]):
    cols: list[str]

    def _fit(self, data_fit: pd.DataFrame) -> pd.Series:
        return data_fit[self.cols].mean()

    def _apply(self, data_apply: pd.DataFrame, state: pd.Series) -> pd.DataFrame:
        means = state
        return data_apply.assign(**{c: data_apply[c] - means[c] for c in self.cols})


# reveal_type(DeMean.FitDeMean)

dmn = DeMean(["price"])
ft = dmn.fit(df)
x = ft.resolved_transform()
result = dmn.fit(df).apply(df)

In [3]:
DeMean.fit?

Signature:
DeMean.fit(
    self: '_Self',
    data_fit: 'Optional[DataIn]' = None,
    bindings: 'Optional[Bindings]' = None,
    backend: 'Optional[Backend]' = None,
) -> 'FitTransform[_Self, DataIn, DataResult]'
Docstring:
Fit this Transform on some data and hyperparam bindings, and return a
:class:`FitTransform` object. The actual return value will be some
subclass of ``FitTransform`` that depends on what class this Transform
is; for example, :meth:`ZScore.fit()` returns a :class:`FitZScore`
object.
File:      ~/frankenfit/src/frankenfit/core.py
Type:      function


In [4]:
result.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [5]:
ff.Transform[pd.DataFrame, pd.DataFrame]._fit
dmn.fit?

Signature:
dmn.fit(
    data_fit: 'Optional[DataIn]' = None,
    bindings: 'Optional[Bindings]' = None,
    backend: 'Optional[Backend]' = None,
) -> 'FitTransform[_Self, DataIn, DataResult]'
Docstring:
Fit this Transform on some data and hyperparam bindings, and return a
:class:`FitTransform` object. The actual return value will be some
subclass of ``FitTransform`` that depends on what class this Transform
is; for example, :meth:`ZScore.fit()` returns a :class:`FitZScore`
object.
File:      ~/frankenfit/src/frankenfit/core.py
Type:      method


In [7]:
ft.apply?

Signature:
ft.apply(
    data_apply: 'Optional[DataIn]' = None,
    backend: 'Optional[Backend]' = None,
) -> 'DataResult'
Docstring: Return the result of applying this FitTransform to the given data.
File:      ~/frankenfit/src/frankenfit/core.py
Type:      method


In [8]:
ft.resolved_transform?

Signature: ft.resolved_transform() -> 'R_co'
Docstring:
Return the Transform that was fit to produce this FitTransform, with all
hyperparameters resolved to their fit-time bindings.
File:      ~/frankenfit/src/frankenfit/core.py
Type:      method


In [20]:
from typing import ClassVar, Type
from abc import abstractmethod

class FitDeMean(ff.FitTransform["DeMean", pd.DataFrame, pd.DataFrame]):
    @abstractmethod
    def apply(
        self,
        data_apply: Optional[pd.DataFrame] = None,
        backend: Optional[ff.Backend] = None,
    ) -> pd.DataFrame:
        """My apply docstr"""
        print("my overridden apply")
        return super().apply(data_apply=data_apply, backend=backend)

def _weighted_means(df, cols, w_col):
    return df[cols].multiply(df[w_col], axis="index").sum() / df[w_col].sum()

@ff.params
class DeMean(ffc.Transform[pd.DataFrame, pd.DataFrame]):
    """
    De-mean some columns.
    """
    cols: list[str]
    w_col: Optional[str] = None

    FitTransformClass: ClassVar[Type[ff.FitTransform]] = FitDeMean


    def _fit(self, data_fit: pd.DataFrame, bindings=None) -> pd.Series:
        if self.w_col is not None:
            return _weighted_means(data_fit, self.cols, self.w_col)
        return data_fit[self.cols].mean()

    def _apply(self, data_apply: pd.DataFrame, state: pd.Series) -> pd.DataFrame:
        means = state
        return data_apply.assign(**{c: data_apply[c] - means[c] for c in self.cols})

    _Self = TypeVar("_Self", bound="DeMean")

    def fit(self: _Self, data_fit: Optional[pd.DataFrame] = None, bindings: Optional[ff.Bindings] = None, backend: Optional[ff.Backend] = None) -> ff.FitTransform[_Self, pd.DataFrame, pd.DataFrame]:
        print("my overridden fit")
        return super().fit(data_fit, bindings, backend)


In [18]:
DeMean.fit?

Signature:
DeMean.fit(
    self: ~_Self,
    data_fit: Optional[pandas.core.frame.DataFrame] = None,
    bindings: Optional[dict[str, Any]] = None,
    backend: Optional[frankenfit.backend.Backend] = None,
) -> frankenfit.core.FitTransform[~_Self, pandas.core.frame.DataFrame, pandas.core.frame.DataFrame]
Docstring:
Fit this Transform on some data and hyperparam bindings, and return a
:class:`FitTransform` object. The actual return value will be some
subclass of ``FitTransform`` that depends on what class this Transform
is; for example, :meth:`ZScore.fit()` returns a :class:`FitZScore`
object.
File:      /tmp/ipykernel_65676/3608603648.py
Type:      function


In [15]:
DeMean.FitTransformClass.apply?

Signature:
DeMean.FitTransformClass.apply(
    self,
    data_apply: Optional[pandas.core.frame.DataFrame] = None,
    backend: Optional[frankenfit.backend.Backend] = None,
) -> pandas.core.frame.DataFrame
Docstring: My apply docstr
File:      /tmp/ipykernel_65676/4061390198.py
Type:      function


In [13]:
DeMean.FitTransformClass?

Init signature:
DeMean.FitTransformClass(
    resolved_transform: 'R_co',
    state: 'Any',
    bindings: 'Optional[Bindings]' = None,
)
Docstring:     
The result of fitting a :class:`{transform_class_name}` Transform. Call this
object's :meth:`apply()` method on some data to get the result of applying the
now-fit transformation.

All parameters of the fit {transform_class_name} are available as instance
variables, with any hyperparameters fully resolved against whatever bindings were
provided at fit-time.

The fit state of the transformation, as returned by {transform_class_name}'s
``_fit()`` method at fit-time, is available from :meth:`state()`, and this is the
state that will be used at apply-time (i.e., passed as the ``state``
argument of the user's ``_fit()`` method).
Type:           type
Subclasses:     


In [14]:
FitDeMean.apply?

Signature:
FitDeMean.apply(
    self,
    data_apply: Optional[pandas.core.frame.DataFrame] = None,
    backend: Optional[frankenfit.backend.Backend] = None,
) -> pandas.core.frame.DataFrame
Docstring: My apply docstr
File:      /tmp/ipykernel_65676/4061390198.py
Type:      function


In [21]:
dmn = DeMean(["price"])
fit_dmn = dmn.fit(df)
print(fit_dmn)
fit_dmn.state()

my overridden fit
FitDeMean(resolved_transform=DeMean(tag='DeMean#4', cols=['price'], w_col=None), state=<class 'pandas.core.series.Series'>, bindings={}, )


price    3932.799722
dtype: float64

In [22]:
fit_dmn

FitDeMean(resolved_transform=DeMean(tag='DeMean#4', cols=['price'], w_col=None), state=<class 'pandas.core.series.Series'>, bindings={}, )

In [23]:
fit_dmn.state?

Signature: fit_dmn.state() -> 'Any'
Docstring:
Return the fit state of the transformation, which is an arbitrary object
determined by the implementation of ``{transform_class_name}._fit()``.
File:      ~/frankenfit/src/frankenfit/core.py
Type:      method


In [24]:
fit_dmn.apply?

Signature:
fit_dmn.apply(
    data_apply: Optional[pandas.core.frame.DataFrame] = None,
    backend: Optional[frankenfit.backend.Backend] = None,
) -> pandas.core.frame.DataFrame
Docstring: My apply docstr
File:      /tmp/ipykernel_65676/4032344889.py
Type:      method


In [25]:
dmn.fit(df, backend=None)

my overridden fit


FitDeMean(resolved_transform=DeMean(tag='DeMean#4', cols=['price'], w_col=None), state=<class 'pandas.core.series.Series'>, bindings={}, )

In [26]:
fit_dmn.apply?

Signature:
fit_dmn.apply(
    data_apply: Optional[pandas.core.frame.DataFrame] = None,
    backend: Optional[frankenfit.backend.Backend] = None,
) -> pandas.core.frame.DataFrame
Docstring: My apply docstr
File:      /tmp/ipykernel_65676/4032344889.py
Type:      method


In [27]:
dmn.fit(df).apply(df, backend=None)

my overridden fit
my overridden apply


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,-1175.799722,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,-1175.799722,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,-1175.799722,5.66,5.68,3.56


In [28]:
dmn_hp = DeMean(ffc.HP("cols"))
dmn_hp.fit(df, bindings={"cols": ["price"]}).apply(df)

my overridden fit
my overridden apply


,carat,cut,color,clarity,depth,table,price,x,y,z
index,,,,,,,,,,
1,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,-1175.799722,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,-1175.799722,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,-1175.799722,5.66,5.68,3.56
